# Imports

In [1]:
import pandas as pd
import nltk
import heapq
import numpy as np
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
from operator import itemgetter 
import itertools

stop_words = stopwords.words('german')

# Rouge n

Rouge steht für Recall-Oriented understudy for gisting Evaluation. Es wird gebraucht um automatisch erzeugte Zusammenfassungen zu evaluieren. Es funktioniert indem es die automatisch generierten Texten nimmt und sie gegen Referenzsummaries vergleicht. 

Machine Summary: the cat was found under the bed

Referenz Summary: the cat was under the bed

### Recall und Precision

Recall in ROUGE sagt einfach gesagt aus wieviel von der Referenz Summary in der Machine Summary ist.  

$$Recall_{} = {{Anzahl überlappende Wörter} \over {Anzahl Wörter in Referenz Summary}}.$$

Im obigen Fall würde das 1 geben. Das ist aber zu genau um wahr zu sein. Darum wird auch noch die Precision gerechnet. Die sagt aus, wieviel der Machine Summary relevant war, oder gebruacht wurde.

$$Precision_{} = {{Anzahl überlappende Wörter} \over {Anzahl Wörter in Machine Summary}}.$$


Dies gibt nun 0.86 und sagt somit aus, dass 6 der 7 Wörter im Machine Summary relevant waren. 


Rouge 1 und Rouge 2 machen grundsätzlich das gleiche. Rouge 1 mit Unigram(bsp von oben) Rouge 2 mit bigrams. zb:

Machine: 

the cat, 
cat was, 
was found, 
found under, 
under the, 
the bed

Referenz:

the cat, 
cat was, 
was under, 
under the, 
the bed


Das ergibt dann folgende Scores: Recall = 0.8,  Precision = 0.67

Wie man sieht ist es nun deutlich ungenauer geworden. 

Der Grund wieso man Rouge1 und Rouge2 verwenden sollte ist weil man so besser die Genauigkeit der Zusammenfassung berehcnen kann.

In [2]:
#Erzeut ngram für Rouge n
def get_n_gram(n,text):
    n_gram = set()
    max_index = len(text) - n
    for i in range(max_index + 1):
        n_gram.add(tuple(text[i:i + n]))
    return n_gram

In [3]:
# Nimmt Sätze und splitet es in einzelne Wörter.
def text_to_words(sentences):
    return list(itertools.chain(*[_.split(" ") for _ in sentences]))

In [4]:
#Erzeugt die ngrams  für mehrere Sätze
def get_word_ngrams(n, sentences):
    words = text_to_words(sentences)
    return get_n_gram(n, words)

In [5]:
#Bereitet die Daten vor und rechnet schliesslich die  Rouge n Metriken
def rouge_n(evaluated_sentences, reference_sentences, n=2):

    evaluated_ngrams = get_word_ngrams(n, evaluated_sentences)
    reference_ngrams = get_word_ngrams(n, reference_sentences)
    reference_count = len(reference_ngrams)
    evaluated_count = len(evaluated_ngrams)

    # Gets the overlapping ngrams between evaluated and reference
    overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
    overlapping_count = len(overlapping_ngrams)

    return f_r_p_rouge_n(evaluated_count, reference_count, overlapping_count)


In [6]:
#Rechnet Rouge Metriken
def f_r_p_rouge_n(evaluated_count, reference_count, overlapping_count):
    #Edge case handling. Falls einen Wert 0 ist. 
    if evaluated_count == 0:
        precision = 0.0
    else:
        precision = overlapping_count / evaluated_count

    if reference_count == 0:
        recall = 0.0
    else:
        recall = overlapping_count / reference_count
    #Falls precision + recall = 0 -> div_zero Error, darum +1e-8. Es verändert den F1 score minimal
    f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

    return f1_score,precision,recall

# Textrank
Der Textrank Algorithmus basiert auf der Pagerank Algorithmus der von Google verwendet wird um Webpages einen Ranking zu geben mit Hilfe der Suchanfrage. 

### Aus Wikiwand
> PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites.

> PageRank is a link analysis algorithm and it assigns a numerical weighting to each element of a hyperlinked set of documents, such as the World Wide Web, with the purpose of "measuring" its relative importance within the set. The algorithm may be applied to any collection of entities with reciprocal quotations and references. The numerical weight that it assigns to any given element E is referred to as the PageRank of E and denoted by PR(E).


https://www.wikiwand.com/en/PageRank

### Es verwendet folgendes
* N Seiten mit Links
* P = PageRank Vektor(P[i] ist Pagerank von i
* A[i][j] ist die Wahrscheinlich von i nach j zu gehen (Wert ist 0-1)


### Pagerank zu Textrak

Die Frage lautet nun, wie kommt man von Pagerank auf Textrank?

Man macht folgendes:
* Wir nehman an, ein Satz ist das gleiche wie eine Page
* Die Wahrscheinlichkeit von Satz A zu Satz B zu gehen ist wie die Ähnlichkeit der beiden Sätzen

Dabei wurde folgendes festgestellt:
* Man muss keinen Model trainieren, es  ist unsupervised
* Der Algorithmus ist Sprachunabhängig, jedoch sollte man trotzdem Stopwords entfernen um eine bessere Accuracy zu erreichen

Diese Bedingungen müssen stets erfüllt sein damit er Algorithmus richtig läuft:
* 0 <= similarity(A, B) <= 1
* similarity(A, A) = 1
* similarity(A, B) =/= 1 falls A =/= B


Um die Ähnlichkeit zwischen 2 Vektoren zu berechnen wird die cosine_distance Funktion von nltk verwendet.

In [7]:
# Pagerank benutzt wahrscheinlichkeiten, welche Webseite als nächstes besucht wird. Da wir das hier nicht haben benutzen wir die Ähnlichkeit zwischen 2 Sätzen und verwenden diese als Ähnlichkeit
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    #Vektor Satz 1
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
    #Vektor Satz 2
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

Nun bruacht man noch die PageRank Übergangsmatrix indem wir die SentenceSimilarity Matrix verwenden.

In [8]:
def build_similarity_matrix(sentences, stopwords=None):
    # Matrix initialisieren mit 0-Werten.
    S = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
    return S

In [9]:
def textrank(sentences, top_n=4, stopwords=None):#top 4 Sätze gewählt damit es im 100Worte bereich lieght +=15%
    S = build_similarity_matrix(sentences, stop_words) 
    sentence_ranks = pagerank(S)
 
    #Sätze nach Wert sortieren
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(sentences)
    return summary

In [10]:
def pagerank(A, eps=0.0001, d=0.85):
    # eps: Algorithmus stoppen wenn Die Differenz zwischen 2 Iterationen kleiner gleich eps ist.
    # d: Dämpfungsfaktor mit Wahrscheinlich 1-d dass eine andere Seite zufällig gewählt wird. 
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P

In [11]:
def summary(text):
    total = []
    for i in text:
        summary  = []
        for idx, sentence in enumerate(textrank(i, stopwords=stopwords.words('german'))):
            summary.append(''.join(sentence))
        total.append(summary)
    return total

# Daten Einlesen

In [12]:
sum_train = pd.read_csv('Summarization_train.csv', delimiter=',')
extracted_text_train = []
extracted_summary_train = []
for i in range(len(sum_train)):
    extracted_text_train.append(nltk.sent_tokenize(sum_train.values[i][0]))
    extracted_summary_train.append(nltk.sent_tokenize(sum_train.values[i][1]))

sum_test = pd.read_csv('Summarization_test.csv', delimiter=',')
extracted_text_test = []
for i in range(len(sum_test)):
    extracted_text_test.append(nltk.sent_tokenize(sum_test.values[i][0]))
    

# Scores berechnen für die ersten 100 Sätzen, da es mit mehr bei mir MemoryError verursacht.

In [13]:
a = summary(extracted_text_train[:100])
f1 = []
precision = []
recall = []

for i in range(len(a)):
    f1cur,precisioncur,recallcur = rouge_n(a[i], extracted_summary_train[i],1)
    f1.append(f1cur)
    precision.append(precisioncur)
    recall.append(recallcur)
    
print('Rouge 1: F1: ', np.mean(f1),' Precision : ',np.mean(precision),' Recall: ',np.mean(recall))

for i in range(len(a)):
    f1cur,precisioncur,recallcur = rouge_n(a[i], extracted_summary_train[i],2)
    f1.append(f1cur)
    precision.append(precisioncur)
    recall.append(recallcur)
    
print('Rouge 2: F1: ', np.mean(f1),' Precision : ',np.mean(precision),' Recall: ',np.mean(recall))

Rouge 1: F1:  0.16047178560124856  Precision :  0.12109520406045457  Recall:  0.27817343659422256
Rouge 2: F1:  0.09190329251135099  Precision :  0.06920316924316924  Recall:  0.16075479619192326


# Summary testen auf Train file

In [14]:
print(a[5])
print()
print(extracted_text_train[5])

['Nachdem das Stimmenergebnis bei der rheinland-pfälzischen Landtagswahl 2016 nicht für eine Fortsetzung der Rot-Grünen Koalition ausgereicht hatte, eine schwarz-gelbe Koalition jedoch ebenfalls keine Mehrheit erzielen konnte und Malu Dreyer eine Grosse Koalition als "Ultima Ratio" befand, einigten sich SPD, FDP und Bündnis 90/Die Grünen auf die Bildung der ersten Ampelkoalition in Rheinland-Pfalz.', 'Auch in Schleswig-Holstein war zuvor eine aufgrund des zweiten Platzes der SPD weniger realistische, aber von den Grünen lange bevorzugte Ampelkoalition nach Sondierungen mit der FDP nicht zu Stande gekommen, so dass dort eine in Deutschland bis dahin noch seltener diskutierte Schwarze Ampel-Koalition unter Führung der CDU gebildet wurde.', 'Nach der Landtags- und Gemeinderatswahl in Wien 1996 wurde eine solche Variante rechnerisch möglich, allerdings wurde seitens der SPÖ eine stabile Zweierkoalition mit ÖVP bevorzugt.', 'Unter dem Begriff traffic light coalition wird in Englan

# Summary testen auf Test file

In [15]:
print(extracted_text_test[50])
print()
summary  = []
for idx, sentence in enumerate(textrank(extracted_text_test[50], stopwords=stopwords.words('german'))):
    summary.append(''.join(sentence))
        
print(summary)

['August Wilhelm Iffland wurde in Hannover im Leibnizhaus als Sohn eines Registrators an der Königlichen Kriegskanzlei geboren.', 'Am dortigen Lyceum war er Mitschüler von Karl Philipp Moritz, der die Begegnung in seinem autobiographischen Roman "Anton Reiser" schilderte.', 'Iffland wurde von seinen angesehenen Eltern für das Studium der Theologie bestimmt, entwich aber 1777 heimlich nach Gotha, wo er Mitglied des Hoftheaters wurde und in Friedrich Wilhelm Gotter einen freundschaftlichen Ratgeber sowie in Conrad Ekhof, Heinrich Beck und Johann David Beil Vorbilder fand.', '1779 mit dem grössten Teil des in Gotha verabschiedeten Schauspielerpersonals von dem Kurfürsten Karl Theodor für die Mannheimer Bühne gewonnen, erwarb sich Iffland hier sowie durch Gastvorstellungen bald einen Namen.', 'Er entwickelte sich zum Charakterdarsteller, der die psychologisch-realistische Schauspielkunst in den Mittelpunkt seiner Arbeit stellte.', 'Einen Triumph erlebte er 1782 als Franz Moor in der